In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import DepthwiseConv2D, Conv2D, BatchNormalization, ReLU, GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.utils import class_weight
from collections import Counter


In [22]:
data_dir = '/content/drive/MyDrive/UCMerced_LandUse/UCMerced_LandUse/Images_converted'
img_height, img_width = 224, 224
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)

In [23]:
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

Found 1680 images belonging to 21 classes.
Found 420 images belonging to 21 classes.


In [24]:
counter = Counter(train_generator.classes)
total = sum(counter.values())
class_weights_dict = {cls: total/(len(counter) * count) for cls, count in counter.items()}

In [25]:
def build_googlenet_with_dwsc(input_shape, num_classes):
    base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = DepthwiseConv2D(kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    return model

input_shape = (img_height, img_width, 3)
num_classes = train_generator.num_classes
model = build_googlenet_with_dwsc(input_shape, num_classes)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [26]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

In [39]:
epochs = 35

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    class_weight=class_weights_dict,
    callbacks=[reduce_lr]
)


Epoch 1/35
53/53 ━━━━━━━━━━━━━━━━━━━━ 27s 463ms/step - accuracy: 0.9340 - loss: 0.2306 - val_accuracy: 0.8548 - val_loss: 0.4395 - learning_rate: 1.0000e-06
Epoch 2/35
53/53 ━━━━━━━━━━━━━━━━━━━━ 26s 437ms/step - accuracy: 0.9270 - loss: 0.2261 - val_accuracy: 0.8476 - val_loss: 0.4239 - learning_rate: 1.0000e-06
Epoch 3/35
53/53 ━━━━━━━━━━━━━━━━━━━━ 27s 446ms/step - accuracy: 0.9191 - loss: 0.2501 - val_accuracy: 0.8548 - val_loss: 0.4368 - learning_rate: 1.0000e-06
Epoch 4/35
53/53 ━━━━━━━━━━━━━━━━━━━━ 26s 442ms/step - accuracy: 0.9094 - loss: 0.2638 - val_accuracy: 0.8500 - val_loss: 0.4007 - learning_rate: 1.0000e-06
Epoch 5/35
53/53 ━━━━━━━━━━━━━━━━━━━━ 26s 436ms/step - accuracy: 0.9000 - loss: 0.2707 - val_accuracy: 0.8381 - val_loss: 0.4691 - learning_rate: 1.0000e-06
Epoch 6/35
53/53 ━━━━━━━━━━━━━━━━━━━━ 26s 440ms/step - accuracy: 0.9220 - loss: 0.2405 - val_accuracy: 0.8333 - val_loss: 0.4276 - learning_rate: 1.0000e-06
Epoch 7/35
53/53 ━━━━━━━━━━━━━━━━━━━━ 26s 439ms/step - acc

In [40]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')


14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 365ms/step - accuracy: 0.8443 - loss: 0.4563
Validation Loss: 0.42566144466400146
Validation Accuracy: 0.8571428656578064


In [44]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay

plt.show()
train_loss, train_accuracy = model.evaluate(train_generator, verbose=1)
print(f"Training Loss: {train_loss}")
print(f"Training Accuracy: {train_accuracy}")


53/53 ━━━━━━━━━━━━━━━━━━━━ 23s 426ms/step - accuracy: 0.9654 - loss: 0.1228
Training Loss: 0.12868759036064148
Training Accuracy: 0.9624999761581421


In [47]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, fbeta_score, cohen_kappa_score, accuracy_score

y_pred = model.predict(validation_generator)
y_true = validation_generator.classes
y_pred_classes = np.argmax(y_pred, axis=1)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')
f2 = fbeta_score(y_true, y_pred_classes, beta=2, average='weighted')
kappa = cohen_kappa_score(y_true, y_pred_classes)
oa = accuracy_score(y_true, y_pred_classes)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")
print(f"F2-score: {f2}")
print(f"Kappa: {kappa}")
print(f"Overall Accuracy (OA): {oa}")


14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 365ms/step
Precision: 0.024295526915664216
Recall: 0.023809523809523808
F1-score: 0.023928617152957077
F2-score: 0.023828529926940168
Kappa: -0.02499999999999991
Overall Accuracy (OA): 0.023809523809523808
